In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir -p /content/ExposureDataset/train
!mkdir -p /content/ExposureDataset/test

!unzip -q "/content/drive/MyDrive/Dataset/training.zip" -d "/content/ExposureDataset/train"
!unzip -q "/content/drive/MyDrive/Dataset/test.zip" -d "/content/ExposureDataset/test"


In [27]:
train_set = ExposureDataset(TRAIN_INPUT, TRAIN_GT)
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)

test_set = ExposureDataset(TEST_INPUT, TEST_GT)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)


In [53]:
!git clone https://github.com/Li-Chongyi/Zero-DCE_extension.git
%cd Zero-DCE_extension/Zero-DCE++


Cloning into 'Zero-DCE_extension'...
remote: Enumerating objects: 2140, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2140 (delta 0), reused 2 (delta 0), pack-reused 2136 (from 1)
Receiving objects: 100% (2140/2140), 100.92 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content/Zero-DCE_extension/Zero-DCE++


In [54]:
!pip install opencv-python matplotlib pillow
!pip install lpips


In [55]:
"""
Re-evaluate Zero-DCE++ on all 750 test images
"""
import torch
import numpy as np
from tqdm import tqdm
import os
import cv2
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import lpips

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load Zero-DCE++ model (make sure you're in the right directory)
from model import enhance_net_nopool

model = enhance_net_nopool(scale_factor=1).to(device)
checkpoint = torch.load("snapshots_Zero_DCE++/Epoch99.pth", map_location=device)
model.load_state_dict(checkpoint)
model.eval()

print("✓ Zero-DCE++ model loaded")

# Initialize LPIPS
lpips_fn = lpips.LPIPS(net='vgg').to(device)
print("✓ LPIPS metric loaded")

def enhance_dce(img):
    with torch.no_grad():
        enhanced_img, _ = model(img.to(device))
    return enhanced_img.clamp(0, 1)

def compute_psnr(pred, gt):
    pred = pred.squeeze().permute(1,2,0).cpu().numpy()
    gt = gt.squeeze().permute(1,2,0).cpu().numpy()
    return peak_signal_noise_ratio(gt, pred, data_range=1.0)

def compute_ssim(pred, gt):
    pred = pred.squeeze().permute(1,2,0).cpu().numpy()
    gt = gt.squeeze().permute(1,2,0).cpu().numpy()
    return structural_similarity(gt, pred, channel_axis=2, data_range=1.0)

def compute_lpips(pred, gt):
    return lpips_fn(pred, gt).item()

# Create test loader (use the same one as U-Net evaluation)
from torch.utils.data import DataLoader

TEST_INPUT = "/content/ExposureDataset/test/INPUT_IMAGES"
TEST_GT = "/content/ExposureDataset/test/GT_IMAGES"

test_set = ExposureDataset(TEST_INPUT, TEST_GT, size=512)
test_loader = DataLoader(
    test_set,
    batch_size=32,  # Same batch size as U-Net
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    prefetch_factor=4
)

print(f"✓ Test set loaded: {len(test_set)} images")

# Create results directory
results_dir = "/content/DCE_results_750"
os.makedirs(results_dir, exist_ok=True)

# Evaluation
psnr_list, ssim_list, lpips_list = [], [], []

print("\n" + "="*60)
print("📊 Evaluating Zero-DCE++ on 750 images")
print("="*60)

with torch.no_grad():
    for batch_idx, (inputs, gt) in enumerate(tqdm(test_loader, desc="Evaluating DCE++")):
        inputs = inputs.to(device)
        gt = gt.to(device)

        # Enhance with DCE++
        pred = enhance_dce(inputs)

        # Process each image in the batch
        for img_idx in range(pred.size(0)):
            # Save prediction image
            img = pred[img_idx].permute(1,2,0).cpu().numpy()
            img = (img * 255).astype(np.uint8)

            global_idx = batch_idx * 32 + img_idx
            cv2.imwrite(f"{results_dir}/{global_idx:04d}.png",
                       cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

            # Compute metrics for this image
            pred_single = pred[img_idx:img_idx+1]
            gt_single = gt[img_idx:img_idx+1]

            psnr_list.append(compute_psnr(pred_single, gt_single))
            ssim_list.append(compute_ssim(pred_single, gt_single))
            lpips_list.append(compute_lpips(pred_single, gt_single))

# Print results
print("\n" + "="*60)
print("📈 Zero-DCE++ Final Metrics (750 images)")
print("="*60)
print(f"  PSNR  : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f}")
print(f"  SSIM  : {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")
print(f"  LPIPS : {np.mean(lpips_list):.4f} ± {np.std(lpips_list):.4f}")
print("="*60)
print(f"\n💾 Results saved to: {results_dir}")
print(f"📊 Total images processed: {len(psnr_list)}")

# Fair comparison with U-Net
print("\n" + "="*60)
print("📊 FAIR Comparison (Both on 750 images)")
print("="*60)
print(f"{'Metric':<10} {'Zero-DCE++':<15} {'U-Net':<15} {'Improvement':<15}")
print("-"*60)
print(f"{'PSNR':<10} {np.mean(psnr_list):<15.4f} {18.4433:<15.4f} {(18.4433-np.mean(psnr_list)):<+15.4f}")
print(f"{'SSIM':<10} {np.mean(ssim_list):<15.4f} {0.7823:<15.4f} {(0.7823-np.mean(ssim_list)):<+15.4f}")
print(f"{'LPIPS':<10} {np.mean(lpips_list):<15.4f} {0.2091:<15.4f} {(0.2091-np.mean(lpips_list)):<+15.4f}")
print("="*60)

Using device: cuda
✓ Zero-DCE++ model loaded
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth
✓ LPIPS metric loaded
✓ Test set loaded: 750 images

📊 Evaluating Zero-DCE++ on 750 images


Evaluating DCE++: 100%|██████████| 24/24 [01:08<00:00,  2.84s/it]


📈 Zero-DCE++ Final Metrics (750 images)
  PSNR  : 11.4899 ± 3.1235
  SSIM  : 0.6684 ± 0.1204
  LPIPS : 0.2189 ± 0.0685

💾 Results saved to: /content/DCE_results_750
📊 Total images processed: 750

📊 FAIR Comparison (Both on 750 images)
Metric     Zero-DCE++      U-Net           Improvement    
------------------------------------------------------------
PSNR       11.4899         18.4433         +6.9534        
SSIM       0.6684          0.7823          +0.1139        
LPIPS      0.2189          0.2091          -0.0098        


# U-Net

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import os
import cv2
import time
import numpy as np
from math import log10
from skimage.metrics import peak_signal_noise_ratio, structural_similarity


In [40]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[32, 64, 128]):
        super(UNet, self).__init__()
        self.pool = nn.MaxPool2d(2)
        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()

        # ------ Encoder ------
        channels = in_channels
        for f in features:
            self.encoders.append(self.conv_block(channels, f))
            channels = f

        # ------ Bottleneck ------
        self.bottleneck = self.conv_block(features[-1], features[-1]*2)

        # ------ Decoder ------
        for f in reversed(features):
            self.decoders.append(
                nn.ConvTranspose2d(features[-1]*2 if f == features[-1] else f*2,
                                 f, kernel_size=2, stride=2)
            )
            self.decoders.append(self.conv_block(f * 2, f))

        # final 1×1 conv
        self.final = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        skips = []

        # Encoder
        for enc in self.encoders:
            x = enc(x)
            skips.append(x)
            x = self.pool(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Reverse skip order
        skips = skips[::-1]

        # Decoder
        for idx in range(0, len(self.decoders), 2):
            x = self.decoders[idx](x)  # upconv
            skip = skips[idx // 2]

            # align shapes if needed
            if x.size()[2:] != skip.size()[2:]:
                x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=True)

            x = torch.cat((skip, x), dim=1)  # concat channels
            x = self.decoders[idx + 1](x)    # conv block

        return torch.sigmoid(self.final(x))


def compute_psnr(pred, gt):
    mse = F.mse_loss(pred, gt).item()
    if mse == 0:
        return 100
    return 10 * log10(1 / mse)



In [39]:
BATCH_SIZE = 32          # ⬆️ Increased from 4 (8x more!)
NUM_WORKERS = 8          # ⬆️ For faster data loading
PREFETCH_FACTOR = 4      # ⬆️ Prefetch batches
PIN_MEMORY = True        # ⬆️ Faster CPU->GPU transfer
IMAGE_SIZE = 512         # Can increase to 768 if you want better quality

EPOCHS = 30
LR = 1e-4
early_stop_patience = 5


In [41]:
# ============================================
# OPTIMIZED DATA LOADING
# ============================================

print("🚀 Creating OPTIMIZED data loaders...")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Num workers: {NUM_WORKERS}")
print(f"   Prefetch factor: {PREFETCH_FACTOR}")

# Training loader with optimizations
train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,          # 🔥 8x larger batches
    shuffle=True,
    num_workers=NUM_WORKERS,        # 🔥 Parallel data loading
    pin_memory=PIN_MEMORY,          # 🔥 Faster transfers
    prefetch_factor=PREFETCH_FACTOR,# 🔥 Prefetch batches
    persistent_workers=True         # 🔥 Keep workers alive
)

# Test loader
test_loader = DataLoader(
    test_set,
    batch_size=BATCH_SIZE,          # 🔥 Can process multiple test images at once
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    prefetch_factor=PREFETCH_FACTOR,
    persistent_workers=True
)

🚀 Creating OPTIMIZED data loaders...
   Batch size: 32
   Num workers: 8
   Prefetch factor: 4


In [44]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n✓ Using device: {device}")

# Initialize model
model = UNet(in_channels=3, out_channels=3, features=[32, 64, 128]).to(device)

# 🔥 Compile model for faster execution (PyTorch 2.0+)
try:
    model = torch.compile(model)
    print("✓ Model compiled with torch.compile() for faster execution")
except:
    print("⚠️  torch.compile() not available, using standard model")

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)  # AdamW is better

# 🔥 Learning rate scheduler for better convergence
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

scaler = GradScaler()

best_loss = float("inf")
patience_counter = 0

# Check GPU utilization
print(f"\n📊 GPU Memory before training:")
print(f"   Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print(f"   Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")

# ============================================
# OPTIMIZED TRAINING LOOP
# ============================================

print("\n" + "="*60)
print("🚀 Starting OPTIMIZED Training")
print("="*60)

for epoch in range(EPOCHS):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    running_psnr = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", ncols=120)

    for inputs, gt in pbar:
        inputs = inputs.to(device, non_blocking=True)  # 🔥 non_blocking for speed
        gt = gt.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)  # 🔥 Slightly faster than zero_grad()

        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, gt)

        scaler.scale(loss).backward()

        # 🔥 Gradient clipping for stability
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        running_psnr += compute_psnr(outputs, gt)

        # Update progress bar
        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "gpu_mem": f"{torch.cuda.memory_allocated()/1e9:.1f}GB"
        })

    epoch_loss = running_loss / len(train_loader)
    epoch_psnr = running_psnr / len(train_loader)
    epoch_time = time.time() - start_time

    # Update learning rate
    scheduler.step(epoch_loss)
    current_lr = optimizer.param_groups[0]['lr']

    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{EPOCHS} Summary:")
    print(f"{'='*60}")
    print(f"  Loss      : {epoch_loss:.4f}")
    print(f"  PSNR      : {epoch_psnr:.2f} dB")
    print(f"  Time      : {epoch_time:.2f} sec ({len(train_loader)*BATCH_SIZE/epoch_time:.1f} img/sec)")
    print(f"  Learn Rate: {current_lr:.6f}")
    print(f"  GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB / {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_unet.pth")
        print(f"  ✅ NEW BEST MODEL SAVED!")
    else:
        patience_counter += 1
        print(f"  ⏳ Patience: {patience_counter}/{early_stop_patience}")
        if patience_counter >= early_stop_patience:
            print("\n⛔ Early stopping activated!")
            break

    print("="*60 + "\n")

print("\n" + "="*60)
print("✅ Training completed!")
print("="*60)

/tmp/ipython-input-1648367814.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



✓ Using device: cuda
✓ Model compiled with torch.compile() for faster execution

📊 GPU Memory before training:
   Allocated: 0.07 GB
   Reserved: 2.77 GB

🚀 Starting OPTIMIZED Training


Epoch 1/30:   0%|                                                                               | 0/553 [00:00<?, ?it/s]/tmp/ipython-input-1648367814.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/30:   0%|                                         | 1/553 [00:20<3:08:48, 20.52s/it, loss=0.2388, gpu_mem=0.3GB]/tmp/ipython-input-1648367814.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/30: 100%|█████████████████████████████████████████| 553/553 [02:23<00:00,  3.84it/s, loss=0.1358, gpu_mem=0.2GB]



Epoch 1/30 Summary:
  Loss      : 0.1611
  PSNR      : 13.97 dB
  Time      : 144.00 sec (122.9 img/sec)
  Learn Rate: 0.000100
  GPU Memory: 0.18 GB / 85.2 GB
  ✅ NEW BEST MODEL SAVED!



Epoch 2/30:   0%|                                                                               | 0/553 [00:00<?, ?it/s]/tmp/ipython-input-1648367814.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 2/30: 100%|█████████████████████████████████████████| 553/553 [01:46<00:00,  5.18it/s, loss=0.1321, gpu_mem=0.2GB]


...


Epoch 28/30 Summary:
  Loss      : 0.1018
  PSNR      : 17.40 dB
  Time      : 108.45 sec (163.2 img/sec)
  Learn Rate: 0.000100
  GPU Memory: 0.18 GB / 85.2 GB
  ✅ NEW BEST MODEL SAVED!



Epoch 29/30: 100%|████████████████████████████████████████| 553/553 [01:46<00:00,  5.17it/s, loss=0.1131, gpu_mem=0.2GB]



Epoch 29/30 Summary:
  Loss      : 0.1015
  PSNR      : 17.43 dB
  Time      : 106.87 sec (165.6 img/sec)
  Learn Rate: 0.000100
  GPU Memory: 0.18 GB / 85.2 GB
  ✅ NEW BEST MODEL SAVED!



Epoch 30/30: 100%|████████████████████████████████████████| 553/553 [01:46<00:00,  5.18it/s, loss=0.1197, gpu_mem=0.2GB]


Epoch 30/30 Summary:
  Loss      : 0.1014
  PSNR      : 17.43 dB
  Time      : 106.76 sec (165.8 img/sec)
  Learn Rate: 0.000100
  GPU Memory: 0.18 GB / 85.2 GB
  ✅ NEW BEST MODEL SAVED!


✅ Training completed!


In [48]:
!pip install lpips
import lpips


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.2 MB/s eta 0:00:00


In [49]:
print("\n🔍 Loading best model for evaluation...")
model.load_state_dict(torch.load("best_unet.pth"))
model.eval()

# Initialize LPIPS
lpips_fn = lpips.LPIPS(net='vgg').to(device)

def compute_psnr_eval(pred, gt):
    """Compute PSNR using skimage for consistency"""
    pred = pred.squeeze().permute(1,2,0).cpu().numpy()
    gt = gt.squeeze().permute(1,2,0).cpu().numpy()
    return peak_signal_noise_ratio(gt, pred, data_range=1.0)

def compute_ssim(pred, gt):
    pred = pred.squeeze().permute(1,2,0).cpu().numpy()
    gt = gt.squeeze().permute(1,2,0).cpu().numpy()
    return structural_similarity(gt, pred, channel_axis=2, data_range=1.0)

def compute_lpips_eval(pred, gt):
    return lpips_fn(pred, gt).item()

# Create results directory
results_dir = "/content/UNet_results"
os.makedirs(results_dir, exist_ok=True)

psnr_list, ssim_list, lpips_list = [], [], []

print("\n📊 Generating predictions and computing metrics...")
print(f"   Batch size: {BATCH_SIZE} (processing {BATCH_SIZE} images at once)")

with torch.no_grad():
    for batch_idx, (inputs, gt) in enumerate(tqdm(test_loader, desc="Evaluating")):
        inputs = inputs.to(device, non_blocking=True)
        gt = gt.to(device, non_blocking=True)

        pred = model(inputs)

        # Process each image in the batch
        for img_idx in range(pred.size(0)):
            # Save prediction image
            img = pred[img_idx].permute(1,2,0).cpu().numpy()
            img = (img * 255).astype(np.uint8)

            global_idx = batch_idx * BATCH_SIZE + img_idx
            cv2.imwrite(f"{results_dir}/{global_idx:04d}.png",
                       cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

            # Compute metrics for this image
            pred_single = pred[img_idx:img_idx+1]
            gt_single = gt[img_idx:img_idx+1]

            psnr_list.append(compute_psnr_eval(pred_single, gt_single))
            ssim_list.append(compute_ssim(pred_single, gt_single))
            lpips_list.append(compute_lpips_eval(pred_single, gt_single))

print("\n" + "="*60)
print("📈 U-Net Final Metrics")
print("="*60)
print(f"  PSNR  : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f}")
print(f"  SSIM  : {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")
print(f"  LPIPS : {np.mean(lpips_list):.4f} ± {np.std(lpips_list):.4f}")
print("="*60)
print(f"\n💾 Results saved to: {results_dir}")
print(f"📊 Total images processed: {len(psnr_list)}")

# Compare with Zero-DCE++
print("\n" + "="*60)
print("📊 Comparison with Zero-DCE++")
print("="*60)
print(f"{'Metric':<10} {'Zero-DCE++':<15} {'U-Net':<15} {'Improvement':<15}")
print("-"*60)
print(f"{'PSNR':<10} {11.4586:<15.4f} {np.mean(psnr_list):<15.4f} {(np.mean(psnr_list)-11.4586):<+15.4f}")
print(f"{'SSIM':<10} {0.6704:<15.4f} {np.mean(ssim_list):<15.4f} {(np.mean(ssim_list)-0.6704):<+15.4f}")
print(f"{'LPIPS':<10} {0.2392:<15.4f} {np.mean(lpips_list):<15.4f} {(np.mean(lpips_list)-0.2392):<+15.4f}")
print("="*60)


🔍 Loading best model for evaluation...
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:02<00:00, 235MB/s]


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth

📊 Generating predictions and computing metrics...
   Batch size: 32 (processing 32 images at once)


Evaluating: 100%|██████████| 24/24 [01:15<00:00,  3.13s/it]


📈 U-Net Final Metrics
  PSNR  : 18.4433 ± 3.2183
  SSIM  : 0.7823 ± 0.0914
  LPIPS : 0.2091 ± 0.0598

💾 Results saved to: /content/UNet_results
📊 Total images processed: 750

📊 Comparison with Zero-DCE++
Metric     Zero-DCE++      U-Net           Improvement    
------------------------------------------------------------
PSNR       11.4586         18.4433         +6.9847        
SSIM       0.6704          0.7823          +0.1119        
LPIPS      0.2392          0.2091          -0.0301        


In [50]:
print(f"Train set size: {len(train_set)}")
print(f"Test set size: {len(test_set)}")
print(f"Train loader batches: {len(train_loader)}")
print(f"Test loader batches: {len(test_loader)}")

Train set size: 17675
Test set size: 750
Train loader batches: 553
Test loader batches: 24
